In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import metrics

train_dir = '../input/vehicle-data-set/cardataset/train'  # Defining directory containing training data
test_dir = '../input/vehicle-data-set/cardataset/test'  # Definining directory containing test data

num_classes = len(os.listdir(train_dir))  # Counting the number of classes in the dataset

height = 150  # Setting the height of the images
width = 150  # Setting the width of the images
channels = 3  # Setting the number of color channels

batch_size = 128  # Defining batch size for training
epochs = 5  # Defining number of epochs for training

# Createing data generators for training and validation data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(height, width),
    batch_size=batch_size,
    seed=1337,
    shuffle=True,
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(height, width), 
    batch_size=batch_size,
    seed=1337,
    shuffle=True,
    class_mode='categorical')

train_num = train_generator.samples  # Getting the number of training samples
validation_num = validation_generator.samples  # Getting the number of validation samples

class Particle:
    """Class representing a particle in the PSO algorithm."""
    def __init__(self, bounds):
        self.position = np.random.uniform(bounds[0], bounds[1], size=len(bounds))
        self.velocity = np.random.uniform(-1, 1, size=len(bounds))
        self.best_position = self.position
        self.best_score = float('inf')

def pso(cost_function, bounds, num_particles, max_iter):
    """Implement Particle Swarm Optimization algorithm."""
    swarm = [Particle(bounds) for _ in range(num_particles)]
    global_best_position = np.zeros_like(bounds[0])
    global_best_score = float('inf')

    for _ in range(max_iter):
        for particle in swarm:
            score = cost_function(particle.position)
            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = particle.position
            if score < global_best_score:
                global_best_score = score
                global_best_position = particle.position

            w = 0.5  # Seting inertia weight
            c1 = 1.5  # Seting cognitive weight
            c2 = 1.5  # Seting social weight
            particle.velocity = (w * particle.velocity) + (c1 * np.random.uniform() * (particle.best_position - particle.position)) + (c2 * np.random.uniform() * (global_best_position - particle.position))
            particle.position += particle.velocity

    return global_best_position

def train_neural_network(params):
    """Train a neural network using the provided hyperparameters."""
    model = Sequential()
    model.add(Conv2D(int(params[0]), (3, 3), input_shape=(height, width, channels)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(int(params[0]), (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(int(params[1]), activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

    history = model.fit_generator(
        train_generator,
        steps_per_epoch=train_num // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_num // batch_size,
        verbose=1)

    return history.history['val_acc'][-1]

bounds = [(16, 64), (64, 512)]  # Defining hyperparameter boundaries
num_particles = 10  # Seting the number of particles in the PSO algorithm
max_iter = 10  # Seting the maximum number of iterations for PSO

best_hyperparameters = pso(train_neural_network, bounds, num_particles, max_iter)

model = Sequential()
model.add(Conv2D(int(best_hyperparameters[0]), (3, 3), input_shape=(height, width, channels)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(int(best_hyperparameters[0]), (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(int(best_hyperparameters[1]), activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

history = model.fit_generator(
    train_generator,
    steps_per_epoch=train_num // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_num // batch_size,
    verbose=1)

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.title('Training and validation accuracy')
plt.plot(epochs, acc, 'red', label='Training acc')
plt.plot(epochs, val_acc, 'blue', label='Validation acc')
plt.legend()
plt.show()

plt.title('Training and validation loss')
plt.plot(epochs, loss, 'red', label='Training loss')
plt.plot(epochs, val_loss, 'blue', label='Validation loss')
plt.legend()
plt.show()


Found 22852 images belonging to 17 classes.
Found 5193 images belonging to 17 classes.
Epoch 1/5
 92/178 [==============>...............] - ETA: 5:26 - loss: 2.0645 - acc: 0.3820